In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder , OneHotEncoder , OrdinalEncoder
from sklearn.model_selection import KFold , StratifiedKFold , train_test_split
from sklearn.linear_model import LinearRegression , LogisticRegression , ElasticNet
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor ,ExtraTreesClassifier , ExtraTreesRegressor , BaggingClassifier , BaggingRegressor , StackingClassifier , StackingClassifier
from sklearn.metrics import roc_auc_score , accuracy_score , classification_report , mean_absolute_error , mean_absolute_percentage_error ,mean_squared_error
from xgboost import XGBClassifier , XGBRegressor

import keras
import torch

In [41]:
train_path = '/content/drive/MyDrive/Hackathons/Cascade Cup/train.csv.zip (Unzipped Files)/train.csv'
test_path = '/content/drive/MyDrive/Hackathons/Cascade Cup/test.csv.zip (Unzipped Files)/test.csv'
submission_path = '/content/drive/MyDrive/Hackathons/Cascade Cup/sample_submission.csv.zip (Unzipped Files)/sample_submission.csv'

In [42]:
df_train  = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_train.sample(5)

,order_time,order_id,order_date,allot_time,accept_time,pickup_time,delivered_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,cancelled,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time,cancelled_time
424121,2021-02-05 15:40:41,185067,2021-02-05 00:00:00,2021-02-05 15:55:47,2021-02-05 15:55:57,2021-02-05 16:03:08,2021-02-05 16:18:30,7497,1.7687,4.15,65.0,65.0,0,0.0,79.0,auto,Auto Reassignment basis Inaction. coreengine.t...,1.0,130.683333,NaN
428513,2021-02-05 16:19:50,189459,2021-02-05 00:00:00,2021-02-05 16:20:37,2021-02-05 16:20:42,2021-02-05 16:30:55,2021-02-05 16:44:28,11383,2.9213,4.04,143.0,143.0,0,0.0,212.0,NaN,NaN,NaN,270.650000,NaN
186376,2021-01-30 15:57:29,425440,2021-01-30 00:00:00,2021-01-30 15:58:17,2021-01-30 15:59:13,2021-01-30 16:27:05,2021-01-30 17:02:33,14889,0.7704,3.67,21.0,21.0,0,0.0,89.0,NaN,NaN,NaN,72.850000,NaN
181452,2021-01-30 14:39:50,420517,2021-01-30 00:00:00,2021-01-30 14:39:51,2021-01-30 14:42:09,2021-01-30 15:03:36,2021-01-30 15:19:56,3575,2.0424,0.76,206.0,206.0,0,0.0,1418.0,NaN,NaN,NaN,304.733333,NaN
6087,2021-01-26 07:57:28,562840,2021-01-26 00:00:00,2021-01-26 07:58:20,2021-01-26 07:58:44,2021-01-26 08:06:10,2021-01-26 08:46:16,1342,0.2562,3.89,111.0,109.0,0,2.0,325.0,NaN,NaN,NaN,147.466667,NaN


In [43]:
df_test

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-02-06 10:03:24,130231,2021-02-06 00:00:00,2021-02-06 10:03:49,2021-02-06 10:04:15,12884,1.658500,4.54,216.0,215.0,1.0,747.0,NaN,NaN,NaN,273.400000
1,2021-02-06 10:03:26,130232,2021-02-06 00:00:00,2021-02-06 10:03:27,2021-02-06 10:03:36,3541,2.070900,5.84,52.0,52.0,0.0,75.0,NaN,NaN,NaN,252.100000
2,2021-02-06 10:03:27,130233,2021-02-06 00:00:00,2021-02-06 10:04:14,2021-02-06 10:05:34,603,1.388400,0.99,289.0,289.0,0.0,2214.0,NaN,NaN,NaN,241.383333
3,2021-02-06 10:03:29,130234,2021-02-06 00:00:00,2021-02-06 10:03:30,2021-02-06 10:03:53,3414,1.903900,2.59,125.0,122.0,3.0,1020.0,NaN,NaN,NaN,291.933333
4,2021-02-06 10:03:35,130235,2021-02-06 00:00:00,2021-02-06 10:03:43,2021-02-06 10:04:43,1426,0.827500,0.94,352.0,350.0,2.0,7284.0,NaN,NaN,NaN,247.133333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144839,2021-02-09 20:13:39,41184,2021-02-09 00:00:00,2021-02-09 20:13:40,2021-02-09 20:13:49,7141,1.417000,3.96,52.0,52.0,0.0,413.0,NaN,NaN,NaN,179.266667
144840,2021-02-09 20:21:36,41185,2021-02-09 00:00:00,2021-02-09 20:21:59,2021-02-09 20:22:33,8113,0.775600,1.61,95.0,93.0,2.0,284.0,NaN,NaN,NaN,244.400000
144841,2021-02-09 20:24:25,41186,2021-02-09 00:00:00,2021-02-09 20:25:03,2021-02-09 20:26:10,5040,2.049400,1.26,77.0,77.0,0.0,119.0,NaN,NaN,NaN,259.000000
144842,2021-02-09 20:28:07,41187,2021-02-09 00:00:00,2021-02-09 20:28:29,2021-02-09 20:28:53,2946,0.080494,5.50,231.0,228.0,3.0,1759.0,NaN,NaN,NaN,814.283333


In [44]:
T = df_train.copy()
T = T[[x for x in T.columns if x in df_test.columns]]
combo = pd.concat([T,df_test],ignore_index = True)
combo

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
0,2021-01-26 02:21:35,556753,2021-01-26 00:00:00,2021-01-26 02:21:59,2021-01-26 02:22:08,11696,1.566600,2.65,46.0,46.0,0.0,621.0,NaN,NaN,NaN,NaN
1,2021-01-26 02:33:16,556754,2021-01-26 00:00:00,2021-01-26 02:33:57,2021-01-26 02:34:45,18117,2.520700,2.76,8.0,8.0,0.0,105.0,NaN,NaN,NaN,3.266667
2,2021-01-26 02:39:49,556755,2021-01-26 00:00:00,2021-01-26 02:39:57,2021-01-26 02:40:13,18623,2.207400,4.80,1.0,1.0,0.0,66.0,NaN,NaN,NaN,9.816667
3,2021-01-26 02:47:53,556756,2021-01-26 00:00:00,2021-01-26 02:48:25,2021-01-26 02:49:06,15945,2.189400,6.38,1.0,1.0,0.0,127.0,NaN,NaN,NaN,17.533333
4,2021-01-26 03:06:30,556757,2021-01-26 00:00:00,2021-01-26 03:07:21,2021-01-26 03:07:57,17589,2.787000,4.01,34.0,34.0,0.0,84.0,NaN,NaN,NaN,1.350000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594839,2021-02-09 20:13:39,41184,2021-02-09 00:00:00,2021-02-09 20:13:40,2021-02-09 20:13:49,7141,1.417000,3.96,52.0,52.0,0.0,413.0,NaN,NaN,NaN,179.266667
594840,2021-02-09 20:21:36,41185,2021-02-09 00:00:00,2021-02-09 20:21:59,2021-02-09 20:22:33,8113,0.775600,1.61,95.0,93.0,2.0,284.0,NaN,NaN,NaN,244.400000
594841,2021-02-09 20:24:25,41186,2021-02-09 00:00:00,2021-02-09 20:25:03,2021-02-09 20:26:10,5040,2.049400,1.26,77.0,77.0,0.0,119.0,NaN,NaN,NaN,259.000000
594842,2021-02-09 20:28:07,41187,2021-02-09 00:00:00,2021-02-09 20:28:29,2021-02-09 20:28:53,2946,0.080494,5.50,231.0,228.0,3.0,1759.0,NaN,NaN,NaN,814.283333


In [45]:
combo.isnull().sum()

order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                225
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders           21721
delivered_orders         22225
undelivered_orders       22225
lifetime_order_count       831
reassignment_method     576468
reassignment_reason     576456
reassigned_order        576456
session_time              8729
dtype: int64

In [46]:
combo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 594844 entries, 0 to 594843
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_time            594844 non-null  object 
 1   order_id              594844 non-null  int64  
 2   order_date            594844 non-null  object 
 3   allot_time            594844 non-null  object 
 4   accept_time           594619 non-null  object 
 5   rider_id              594844 non-null  int64  
 6   first_mile_distance   594844 non-null  float64
 7   last_mile_distance    594844 non-null  float64
 8   alloted_orders        573123 non-null  float64
 9   delivered_orders      572619 non-null  float64
 10  undelivered_orders    572619 non-null  float64
 11  lifetime_order_count  594013 non-null  float64
 12  reassignment_method   18376 non-null   object 
 13  reassignment_reason   18388 non-null   object 
 14  reassigned_order      18388 non-null   float64
 15  

In [47]:
# Filling NAN values of alloted orders
print('null_values in alloted_orders: ',combo['alloted_orders'].isnull().sum())
print()
ckpt = combo[['order_id','rider_id','delivered_orders','undelivered_orders'
,'alloted_orders','reassignment_method',
'reassignment_reason']][combo['alloted_orders'].isnull()==True ]
ckpt_alloted = ckpt[ckpt['delivered_orders'].isnull()==True]
ckpt_alloted = ckpt_alloted[ckpt_alloted['undelivered_orders'].isnull()==True]
idx_lst = list(ckpt_alloted.index)
for index in idx_lst:
  combo['alloted_orders'][index] = 0
  combo['delivered_orders'][index] = 0
  combo['undelivered_orders'][index] = 0





print('null_values in data: ',combo.isnull().sum())
print()

null_values in alloted_orders:  21721



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


null_values in data:  order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                225
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders               0
delivered_orders           504
undelivered_orders         504
lifetime_order_count       831
reassignment_method     576468
reassignment_reason     576456
reassigned_order        576456
session_time              8729
dtype: int64



In [48]:
# Filling remaining NAN values in delivered_orders and undelivered_orders
ckpt2 = combo[combo['delivered_orders'].isnull()==True]
idx_lst2 = list(ckpt2.index)
for index in idx_lst2:
  combo['delivered_orders'][index] = combo['alloted_orders'][index]/2
  combo['undelivered_orders'][index] = combo['alloted_orders'][index]/2
print('null_values in data: ',combo.isnull().sum())
print()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


null_values in data:  order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                225
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders               0
delivered_orders             0
undelivered_orders           0
lifetime_order_count       831
reassignment_method     576468
reassignment_reason     576456
reassigned_order        576456
session_time              8729
dtype: int64



In [49]:
rider_df = combo[['rider_id','lifetime_order_count']]
rider_df = rider_df.fillna(0)
rider_df = pd.DataFrame(rider_df.groupby(['rider_id'])['lifetime_order_count'].mean() ,index = list(range(21567)))
rider_df['rider_id'] = 0
rider_df['rider_id'] = list(range(21567))
rider_df

,lifetime_order_count,rider_id
0,8734.0,0
1,1380.0,1
2,1394.0,2
3,596.0,3
4,306.0,4
...,...,...
21562,0.0,21562
21563,282.0,21563
21564,55.0,21564
21565,31.0,21565


In [52]:
#Filling NAN in lifetime_order count
ckpt3 = combo[combo['lifetime_order_count'].isnull()==True]
nan_rider = ckpt3['rider_id'].values.tolist()
idx_lst3 = list(ckpt3.index)

# rider_dict = {}
# for item in zip(nan_rider,idx_lst3):
#   rider_dict[item[0]] = item[1]

# rider_dict[81]




In [53]:
for item in zip(nan_rider,idx_lst3):
  combo['lifetime_order_count'][item[1]] = rider_df['lifetime_order_count'][rider_df['rider_id'] == item[0]].values

combo.isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                225
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders               0
delivered_orders             0
undelivered_orders           0
lifetime_order_count         0
reassignment_method     576468
reassignment_reason     576456
reassigned_order        576456
session_time              8729
dtype: int64

In [54]:
ckpt4 = 

,order_time,order_id,order_date,allot_time,accept_time,rider_id,first_mile_distance,last_mile_distance,alloted_orders,delivered_orders,undelivered_orders,lifetime_order_count,reassignment_method,reassignment_reason,reassigned_order,session_time
11054,2021-01-26 09:48:13,567807,2021-01-26 00:00:00,2021-01-26 09:48:14,2021-01-26 09:49:29,14345,0.392400,1.46,1.0,0.5,0.5,NaN,NaN,NaN,NaN,221.083333
12097,2021-01-26 11:03:57,568850,2021-01-26 00:00:00,2021-01-26 11:04:45,2021-01-26 11:05:00,21330,0.034309,1.51,1.0,0.5,0.5,NaN,NaN,NaN,NaN,49.416667
13738,2021-01-26 13:06:02,570491,2021-01-26 00:00:00,2021-01-26 13:06:03,2021-01-26 13:06:35,21353,0.043488,6.40,0.0,0.0,0.0,NaN,NaN,NaN,NaN,69.950000
13885,2021-01-26 13:15:50,570638,2021-01-26 00:00:00,2021-01-26 13:16:35,2021-01-26 13:16:53,21354,1.853000,2.11,0.0,0.0,0.0,NaN,NaN,NaN,NaN,52.500000
19958,2021-01-26 14:27:46,576712,2021-01-26 00:00:00,2021-01-26 14:43:54,2021-01-26 14:44:22,21354,0.486800,0.89,0.0,0.0,0.0,NaN,auto,Auto Reassignment basis Inaction. coreengine.t...,1.0,110.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594427,2021-02-09 17:55:09,40772,2021-02-09 00:00:00,2021-02-09 17:55:12,2021-02-09 17:55:19,6271,0.659500,2.59,0.0,0.0,0.0,NaN,NaN,NaN,NaN,144.600000
594544,2021-02-09 17:57:02,40888,2021-02-09 00:00:00,2021-02-09 17:57:14,2021-02-09 17:57:23,8080,1.562300,3.21,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
594551,2021-02-09 17:57:09,40896,2021-02-09 00:00:00,2021-02-09 17:57:11,2021-02-09 17:57:33,8377,0.041071,1.58,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
594660,2021-02-09 17:58:30,41005,2021-02-09 00:00:00,2021-02-09 17:58:31,2021-02-09 17:59:00,728,0.635400,4.07,0.0,0.0,0.0,NaN,NaN,NaN,NaN,347.366667


In [90]:

#Imputing for session time
ckpt4 = combo[combo['session_time'].isnull()==True]
nan_rider_ses = ckpt4['rider_id'].values.tolist()
idx_lst4 = list(ckpt4.index)

#creating a data frame so that imputation will be easy
rider_df_ses = combo[['rider_id','session_time']]
rider_df_ses = rider_df_ses.fillna(0)
rider_df_ses = pd.DataFrame(rider_df_ses.groupby(['rider_id'])['session_time'].mean() ,index = list(range(21567)))
rider_df_ses['rider_id'] = 0
rider_df_ses['rider_id'] = list(range(21567))


#Imputing average session time for a particular rider id
for item in zip(nan_rider_ses,idx_lst4):
  combo['session_time'][item[1]] = rider_df_ses['session_time'][rider_df_ses['rider_id'] == item[0]].values

combo.isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


order_time                   0
order_id                     0
order_date                   0
allot_time                   0
accept_time                225
rider_id                     0
first_mile_distance          0
last_mile_distance           0
alloted_orders               0
delivered_orders             0
undelivered_orders           0
lifetime_order_count         0
reassignment_method     576468
reassignment_reason     576456
reassigned_order        576456
session_time                 0
dtype: int64

In [87]:
rider_df_ses = combo[['rider_id','session_time']]
rider_df_ses = pd.DataFrame(rider_df_ses.groupby(['rider_id'])['session_time'].mean())
rider_df_ses

,session_time
rider_id,
0,348.641667
1,527.734188
2,470.326603
3,350.119608
4,79.933333
...,...
21562,9.816667
21563,11.950000
21564,3.066667


In [ ]:
ck

In [83]:
combo['session_time']

0                NaN
1           3.266667
2           9.816667
3          17.533333
4           1.350000
             ...    
594839    179.266667
594840    244.400000
594841    259.000000
594842    814.283333
594843    161.533333
Name: session_time, Length: 594844, dtype: float64